In [1]:
import tensorflow as tf
import pickle
import numpy as np
import time
import load_cifar_template as lct

In [2]:
training_features_1,training_labels_1 = pickle.load(open('preprocess_batch_1.p','rb'))
training_features_2,training_labels_2 = pickle.load(open('preprocess_batch_2.p','rb'))
training_features_3,training_labels_3 = pickle.load(open('preprocess_batch_3.p','rb'))
training_features_4,training_labels_4 = pickle.load(open('preprocess_batch_4.p','rb'))
training_features_5,training_labels_5 = pickle.load(open('preprocess_batch_5.p','rb'))

valid_features, valid_labels = pickle.load(open('preprocess_validation.p','rb'))
test_features, test_labels = pickle.load(open('preprocess_testing.p','rb'))

In [3]:
train_features = np.vstack([training_features_1, training_features_2, training_features_3, 
                           training_features_4, training_features_5])
train_labels = np.vstack([training_labels_1, training_labels_2, training_labels_3, 
                         training_labels_4, training_labels_5])

In [4]:
tf.reset_default_graph()

<h1>Hyperparameter</h1>

In [5]:
#learning rate
lr = 0.001
#number of epochs
epochs=10
#number of batch_size
batch_size=64

num_classes = 10

In [6]:
he_init=tf.contrib.layers.variance_scaling_initializer()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



<h1>Placeholder</h1>

In [7]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.int64, [None, num_classes], name="Y")

<h1>LeNet-5</h1>

In [8]:
def CNN(X):
    # Here we defind the CNN architecture (LeNet-5)

    # Reshape input to 4-D vector
    input_layer = tf.reshape(X, [-1, 32, 32, 3]) # -1 adds minibatch support.

    # Convolutional Layer #1
    # Has a default stride of 1
    # Output: 28 * 28 * 6
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=6, # Number of filters.
      kernel_size=5, # Size of each filter is 5x5.
      padding="valid", # No padding is applied to the input.
      activation=tf.nn.relu)

    # Pooling Layer #1
    # Sampling half the output of previous layer
    # Output: 14 * 14 * 6
    pool1 = tf.layers.average_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Output: 10 * 10 * 16
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16, # Number of filters
      kernel_size=5, # Size of each filter is 5x5
      padding="valid", # No padding
      activation=tf.nn.relu)

    # Pooling Layer #2
    # Output: 5 * 5 * 16
    pool2 = tf.layers.average_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Reshaping output into a single dimention array for input to fully connected layer
    #pool2_flat = tf.reshape(pool2, [-1, 5 * 5 * 16])
    pool2_flat = tf.layers.flatten(pool2)

    # Fully connected layer #1: Has 120 neurons
    dense1 = tf.layers.dense(inputs=pool2_flat, units=120, activation=tf.nn.tanh,kernel_initializer=he_init)

    # Fully connected layer #2: Has 84 neurons
    dense2 = tf.layers.dense(inputs=dense1, units=84, activation=tf.nn.tanh,kernel_initializer=he_init)

    # Output layer, 10 neurons for each digit
    logits = tf.layers.dense(inputs=dense2, units=10, kernel_initializer=he_init)

    return logits

<h1>Cost and Optimization</h1>

In [9]:
logits = CNN(X)
softmax = tf.nn.softmax(logits)

# Convert our labels into one-hot-vectors
#labels = tf.one_hot(indices=tf.cast(Y, tf.int32), depth=10)

# Compute the cross-entropy loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=Y))

# Use adam optimizer to reduce cost
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost)


# For testing and prediction
predictions = tf.argmax(softmax, axis=1)
correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Initialize all the variables
init = tf.global_variables_initializer()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [10]:
with tf.Session() as sess:
 
    sess.run(init)

    for epoch in range(epochs):
        
        start = time.time()
        
        num_samples = train_features.shape[0]
        num_batches = (num_samples // batch_size) + 1
        epoch_cost = 0.
        
        # Shuffle training data each epoch
        shuffle_index = np.random.permutation(num_samples)
        train_features_shuffled = train_features[shuffle_index]
        train_labels_shuffled = train_labels[shuffle_index]
        
        i = 0
        while i < num_samples:
            batch_x = train_features_shuffled[i:i+batch_size,:]
            batch_y = train_labels_shuffled[i:i+batch_size]

            i += batch_size

            # Train on batch and get back cost
            _, c = sess.run([train_op, cost], feed_dict={X:batch_x, Y:batch_y})
            epoch_cost += (c/num_batches)
            
            #print(epoch_cost)
            
        end = time.time()
        print("Took %f s" % ((end - start)))

        # Get accuracy for validation
        valid_accuracy = accuracy.eval(
            feed_dict={X:valid_features, Y:valid_labels})

        print ("Epoch {}: Cost: {}".format(epoch+1, epoch_cost))
        print("Validation accuracy: {}".format(valid_accuracy))

    test_accuracy = accuracy.eval(feed_dict={X:test_features, Y:test_labels})
    
    print("Testing accuracy: {}".format(test_accuracy))

Took 16.806282 s
Epoch 1: Cost: 1.667781287973577
Validation accuracy: 0.47440001368522644
Took 16.837348 s
Epoch 2: Cost: 1.4240838365459996
Validation accuracy: 0.4851999878883362
Took 16.923164 s
Epoch 3: Cost: 1.3416528298773542
Validation accuracy: 0.527400016784668
Took 16.849493 s
Epoch 4: Cost: 1.2674140223217294
Validation accuracy: 0.5371999740600586
Took 16.893342 s
Epoch 5: Cost: 1.2127844030037538
Validation accuracy: 0.5613999962806702
Took 16.936068 s
Epoch 6: Cost: 1.1634912009096954
Validation accuracy: 0.5842000246047974
Took 16.954651 s
Epoch 7: Cost: 1.1180413997816774
Validation accuracy: 0.5834000110626221
Took 16.942995 s
Epoch 8: Cost: 1.079606527885931
Validation accuracy: 0.5830000042915344
Took 16.910387 s
Epoch 9: Cost: 1.0453544561751191
Validation accuracy: 0.5982000231742859
Took 16.889115 s
Epoch 10: Cost: 1.0147906497798183
Validation accuracy: 0.6092000007629395
Testing accuracy: 0.6133000254631042
